In [8]:
import re
from collections import deque
import glob
def parse_summary_stats(text):
    result = {}
    patterns = {
        'GFLOPs': r"GFLOPs.*: (\d+\.\d+)",
        'Utilization': r"Utilization: (\d+\.\d+)",
        'Cycles': r"Cycles: (\d+)",
        'Energy': r"Energy: (\d+\.\d+) uJ",
        'Area': r"Area: (\d+\.\d+) mm",
        'Algorithmic Computes': r"Algorithmic Computes = (\d+)",
#         'Total_pJ_Algorithmic_Compute': r"Total                        \s+=\s+(\d+\.\d+)",
        'Actual Computes': r"Actual Computes = (\d+)",
#         'Total_pJ_Actual_Compute': r"Total\s+=\s+(\d+\.\d+)"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            result[key] = float(match.group(1))
    
    pattern = r'Total\s*=\s*([\d.]+)'
    totals = re.findall(pattern, text)

    pj_algorithmic_compute = float(totals[0])
    pj_actual_compute = float(totals[1])
    
    result["pj_algorithmic_compute"] = pj_algorithmic_compute
    result["pj_actual_compute"] = pj_actual_compute
    return result

def read_from_summary_stats(filename):
    summary_stats_line = "Summary Stats"
    lines_since_summary_stats = None

    with open(filename, 'r') as file:
        for line in file:
            if summary_stats_line in line:
                lines_since_summary_stats = deque(maxlen=100)  # Adjust maxlen to the maximum number of lines you expect after "Summary Stats"
            if lines_since_summary_stats is not None:
                lines_since_summary_stats.append(line)
    
    if lines_since_summary_stats:
        return list(lines_since_summary_stats)
    else:
        return []

In [17]:
import pandas as pd
archs = ["04.2.2-eyeriss-like-gating-mapspace-search","04.2.3-eyeriss-like-onchip-compression"]
layers = ["sparse","dense"]
for arch in archs:
    for layer_indicator in layers:
    direc = "/home/workspace/lab3/sparse_arch/{}/{}_layers/".format(arch,layer_indicator)

        for layer in glob.glob(direc+"**/dense_pointNet_*", recursive=True):
        #     layer = glob.glob(direc+"**/sparse_pointNet_layer9*",recursive=True)[0]
            # print(layer)
            filename = "{}/timeloop-mapper.stats.txt".format(layer)
            lines_from_summary_stats = read_from_summary_stats(filename)
            text = ''.join(lines_from_summary_stats)
            # print(text)
            # Use the parse_summary_stats function from the previous answer
            parsed_values = parse_summary_stats(text)


            df = pd.DataFrame(parsed_values,index=[0])
            # print("Doing",filename)
            output_file = "{}/summary_output.csv".format(layer)
            df.to_csv(output_file,index=False)
            # print(df)


In [22]:
archs = ["04.2.2-eyeriss-like-gating-mapspace-search","04.2.3-eyeriss-like-onchip-compression"]
layers = ["sparse","dense"]
for arch in archs:
    for layer_indicator in layers:
        direc = "/home/workspace/lab3/sparse_arch/{}/{}_layers/".format(arch,layer_indicator)

        for layer in glob.glob(direc+"**/{}_pointNet_*".format(layer_indicator), recursive=True):
            filename = "{}/timeloop-mapper.ART.yaml".format(layer)
            with open(filename, 'r') as file:
                content = file.read()
            
            pattern = r'area:\s*([\d.]+)'
            areas = re.findall(pattern, content)
            print(layer,areas)
            total_area = sum(float(area) for area in areas)
            area = {"Area":total_area}
            df = pd.DataFrame(area,index=[0])
            output_file = "{}/area.csv".format(layer)
            df.to_csv(output_file,index=False)
#             print("Total area:", total_area)

/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_pointNet_layer1 ['0', '0', '180.0', '3474.22', '537.18', '1239.5', '641667.98994']
/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_pointNet_layer10 ['0', '0', '180.0', '3474.22', '537.18', '1239.5', '641667.98994']
/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_pointNet_layer11 ['0', '0', '180.0', '3474.22', '537.18', '1239.5', '641667.98994']
/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_pointNet_layer12 ['0', '0', '180.0', '3474.22', '537.18', '1239.5', '641667.98994']
/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_pointNet_layer2 ['0', '0', '180.0', '3474.22', '537.18', '1239.5', '641667.98994']
/home/workspace/lab3/sparse_arch/04.2.2-eyeriss-like-gating-mapspace-search/sparse_layers/sparse_

In [11]:
import pandas as pd
df = pd.DataFrame(parsed_values,index=[0])
df

,GFLOPs,Utilization,Cycles,Energy,Area,Algorithmic Computes,Actual Computes,pj_algorithmic_compute,pj_actual_compute
0,127.82,0.38,734004.0,1593.23,0.0,134217728.0,46976205.0,11.87,33.92
